In [1]:
import pandas as pd
import csv
import json
import datetime as dt
import os
from psaw import PushshiftAPI
import itertools
import sys

In [2]:
api = PushshiftAPI()

## find top frequency subreddits outside r/GreatApes

In [2]:
## download 1-column dataframe about what authors have already been downloaded
authors_downloaded = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/authors_downloaded.csv')

In [3]:
authors_downloaded = list(authors_downloaded['Authors'])

In [4]:
## create empty dataframe to collect username, time of post, and subreddit of this post
df_subreddits = pd.DataFrame(columns=["Username","Time","Subreddit"])
df_subreddits.to_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_topSubreddits.csv',index=False)

In [6]:
## go through all authors that have been downloaded and extract the time posted and subreddit of each of their posts
for author in authors_downloaded:

    df_subreddits = pd.DataFrame(columns=["Username","Time","Subreddit"])
    print(df_subreddits)
    
    file_name = r'D:/ISI REU Python/Authors PSAW (Copy)/'+author+'_submissions.json' ## go through submissions
    with open(file_name,'r') as json_file:
        print('submissions for '+author+' starting...')
        for line in json_file:
            try:
                new_line = json.loads(line)
                df_subreddits = df_subreddits.append({"Username":author,"Time":datetime.datetime.fromtimestamp(new_line['created_utc']),"Subreddit":new_line['subreddit']},ignore_index=True)
            except:
                print('user ' + author + ' has no submissions')
                continue
    
    file_name = r'D:/ISI REU Python/Authors PSAW (Copy)/'+author+'_comments.json'  ## go through comments
    with open(file_name,'r') as json_file:
        print('comments for '+author+' starting...')
        for line in json_file:
            try:
                new_line = json.loads(line)
                df_subreddits = df_subreddits.append({"Username":author,"Time":datetime.datetime.fromtimestamp(new_line['created_utc']),"Subreddit":new_line['subreddit']},ignore_index=True)
            except:
                print('user ' + author + ' has no comments')
                continue

    df_subreddits.to_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_topSubreddits.csv',index=False,mode='a',header=False)
    
    print(author + ' done')

In [31]:
df = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_topSubreddits.csv')  ## this is the dataframe that 
                                                                                  ## holds the data from the above code

In [32]:
## look through previously-created dataframe consisting of each GreatApes  users' username and date of first post
## WITHIN GreatApes (not the first post on Reddit overall)
df_GreatApes = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_firstpost.csv')
df_GreatApes = df_GreatApes.drop(['Time (First Post)'], axis=1)  ## delete superfluous index column

In [33]:
df = df.merge(df_GreatApes, on='Username')  ## combine the data of the GreatApes users' first posts AND the subreddits/times they 
                                            ## posted in

In [34]:
## calculate number of days elapsed between first post date and date of later posts
df['Time Difference (Days)'] = (pd.to_datetime(df['Time']) - pd.to_datetime(df['Datetime (First Post)'])).dt.days

In [35]:
## divide into 10 day chunks
df['Time Difference (10 Days)'] = (df['Time Difference (Days)']/10).round(decimals=0)

In [36]:
## sort values based on time difference (days)
df = df.sort_values(['Time Difference (Days)'])
df = df.reset_index()
df = df.drop(['index'],axis=1)

KeyError: "['level_0'] not found in axis"

In [37]:
df = df.drop('index',axis=1)

In [41]:
## split up this dataframe based on the posts posted BEFORE the users' first post in GreatApes vs. after
index = df[df['Time Difference (Days)'] == 0].index.values
for i in index:
    index_int = int(i)
    break
df_1 = df.iloc[:index_int,:]
df_2 = df.iloc[index_int:,:]

In [43]:
df_1.sort_values('Subreddit')

,Username,Time,Subreddit,Datetime (First Post),Time Difference (Days),Time Difference (10 Days)
2982878,RonaldTheRight,2014-05-10 08:40:40,07exchange,2014-05-14 18:04:51,-5,-0.0
306322,ZankerH,2012-04-21 06:14:35,0x10c,2014-09-06 02:58:31,-868,-87.0
345545,ZankerH,2012-06-05 05:49:29,0x10c,2014-09-06 02:58:31,-823,-82.0
494863,ZankerH,2012-10-13 13:48:14,0x10c,2014-09-06 02:58:31,-693,-69.0
683844,ZankerH,2013-02-15 12:08:49,0x10c,2014-09-06 02:58:31,-568,-57.0
...,...,...,...,...,...,...
1789700,Bullshardt,2014-02-11 12:09:56,zyzz,2014-08-14 10:29:40,-184,-18.0
2747871,Chingchonglinglong69,2014-06-25 22:55:02,zyzz,2014-07-20 16:13:14,-25,-2.0
2740605,Chingchonglinglong69,2014-06-25 12:39:22,zyzz,2014-07-20 16:13:14,-26,-3.0
1758208,alex25400,2014-02-28 18:53:44,NaN,2014-09-06 23:02:39,-191,-19.0


In [45]:
## group the dataframe by similar subreddits, counting only the unique number of users who have posted in them
df_1 = df_1.groupby('Subreddit').nunique()

In [46]:
## drop the listed y-axes to make df_1 easier to look at
df_1 = df_1.drop(['Time','Datetime (First Post)','Time Difference (Days)','Time Difference (10 Days)'],axis=1)

In [47]:
df_1

,Username
Subreddit,
07exchange,1
0x10c,4
1000movies,1
1000thworldproblems,3
100BlackMen,2
...,...
zooeydeschanel,3
zookeeperbattle,2
zoology,2


In [48]:
df_1 = df_1.sort_values(by=['Username'],ascending=False)

In [49]:
## split into the top 200 vs. the rest
df_top = df_1.iloc[:200,:]
df_bottom = df_1.iloc[200:,:]

In [58]:
df_top

,Subreddit,Username
0,AskReddit,1980
1,funny,1780
2,pics,1730
3,videos,1539
4,WTF,1524
...,...,...
195,dadjokes,124
196,Shitty_Car_Mods,123
197,tf2,122
198,skeptic,121


In [56]:
df_1.to_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_topSubreddits_freq.csv')

In [51]:
df_top.to_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_topSubreddits_top200 (1).csv')

## download all users in top 200 subreddits

In [3]:
df_top = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_topSubreddits_top200 (1).csv')

In [4]:
top_subreddits = list(df_top['Subreddit'])

In [5]:
## end_epoch is the date of the last post ever posted in GreatApes (calculated in an earlier code)
end_epoch = 1523935718

In [14]:
for i in top_subreddits:
    
    print(i+' starting...')
    counter = 0
    
    results_sub = []
    results_set = set()  ## create set of people who have already been downloaded
    
    gen_sub = api.search_comments(before=end_epoch,subreddit=i)  ## use psaw API

    for line in itertools.islice(gen_sub,None):
        to_append = set()

        if 'author' in line.d_:  ## get the author from the psaw dictionary of each post
            results_set.add(line.d_['author'])
            
        if counter % 10000 == 0 and counter != 0:  ## only download the new data every 10,000 posts
            file_name = r'D:/ISI REU Python/PSAW Data/Authors Lists/top200_authors.csv'
            df_csv = pd.read_csv(file_name)
            authors_listed = set(df_csv['Authors'])
            to_append = results_set - authors_listed  ## only append the authors who haven't been listed yet

            df = pd.DataFrame({'Authors':list(to_append)})

            df.to_csv(file_name,mode='a',header=False)  ## use mode = 'a' to append onto csv
            results_sub = []
            print('10,000 lines from '+i+' done')
        counter += 1
        
    ## after the code runs through the n*10,000 posts, use the following chunk to ensure that the remainder
    ## of posts is accounted for
    file_name = r'D:/ISI REU Python/PSAW Data/Authors Lists/top200_authors.csv'
    df_csv = pd.read_csv(file_name)
    authors_listed = set(df_csv['Authors'])
    to_append = results_set - authors_listed

    df = pd.DataFrame({'Authors':list(to_append)})

    df.to_csv(file_name,mode='a',header=False)

    print(i + ' all done')

In [32]:
## unimportant code

# result_out = pd.read_csv(r'D:/ISI REU Python/PSAW Data/Authors Lists/top200_authors.csv')

# index = result_out[result_out['Authors'] == 'iTube5211'].index.values
# for i in index:
#     index_int = int(i)
# out_1 = result_out.iloc[:index_int+1,:]
# out_2 = result_out.iloc[index_int+1:,:]

In [43]:
## drop unnecessary index axis
out_1.drop(columns=out_1.columns[0], 
        axis=1, 
        inplace=True)

C:\Users\stpau\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [44]:
out_1

,Authors
0,blitzinc43
1,birdpooponfire
2,HowDoYouHearHeavy
3,FormingBadHabits
4,Vivec-Warrior-Poet
...,...
4987210,BigDaddyG4me
4987211,dark-pinoy
4987212,jaksonfivyo
4987213,Rustlingrose


In [45]:
out_1.to_csv(r'D:/ISI REU Python/PSAW Data/Authors Lists/top200_authors.csv')

## calculate earliest post per control user in top 200 subreddits

In [4]:
df_authors = pd.read_csv(r'D:/ISI REU Python/PSAW Data/Authors Lists/top200_authors.csv')
authors_list = df_authors['Authors']
authors_set = set(authors_list)

In [5]:
file_paths_s = []

for root, directories, files in os.walk(r'D:/ISI REU Python/Authors PSAW (Copy)/'):
    for filename in files:
        filepath = os.path.join(root, filename)
        if '_submissions.json' in filepath:
            new_filepath = filepath.replace(r'D:/ISI REU Python/Authors PSAW (Copy)/','').replace('_submissions.json','')
            file_paths_s.append(new_filepath)

file_paths_s = set(dict.fromkeys(file_paths_s))

In [6]:
file_paths_c = []

for root, directories, files in os.walk(r'D:/ISI REU Python/Authors PSAW (Copy)/'):
    for filename in files:
        filepath = os.path.join(root, filename)
        if '_comments.json' in filepath:
            new_filepath = filepath.replace(r'D:/ISI REU Python/Authors PSAW (Copy)/','').replace('_comments.json','')
            file_paths_c.append(new_filepath)

file_paths_c = set(dict.fromkeys(file_paths_c))

In [7]:
file_paths = file_paths_s.intersection(file_paths_c)
set_200authors = authors_set.intersection(file_paths)

In [10]:
df_GreatApes = pd.read_csv(r'D:/ISI REU Python/PSAW Data/Full Data/GreatApes_authors.csv',names=['Authors'])
set_GreatApes = set(df_GreatApes['Authors'])

In [11]:
list_200authors = list(set_200authors - set_GreatApes)

In [12]:
print(list_200authors)

In [13]:
print(len(list_200authors))
print(len(set_200authors))

1101
3139


### filter out users who have not used a hate word

In [21]:
hate_users = []

for author in list_200authors:
    vernacular = set()
    
    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_submissions.json"
    df_s = pd.read_csv(file_name)
    for line in list(df_s['selftext']):
        vernacular.add(line.split())
    for line in list(df_s['title']):
        vernacular.add(line.split())
    
    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_comments.json"
    df_c = pd.read_csv(file_name)
    
    for line in list(df_s['body']):
        vernacular.add(line.split())
        
    for word in hate_words:
        if word in vernacular:
            hate_users.append(author)

In [21]:
# authors_error = []
error_again = []

In [22]:
# set_top200 = set(top_subeddits)
df_top200 = pd.DataFrame(columns = ['Username','Time (First Post)'])  # create dataframe
# df_top200.to_csv(r'D:/ISI REU Python/Analyzed_Data.csv',mode='w',header=True)

counter = 0

# for author in list_200authors:
for author in hate_users:
    
    try:
    
        posted_times_s = []
        posted_times_c = []

    #     if the line in GreatApes, append to temporary times list
#         file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_submissions.json"
#         with open(file_name,'r') as json_file:
#             for line in json_file:
#     #             try:
#                 new_line = json.loads(line)
#     #             if 'subreddit' in new_line.keys():
#     #                 if new_line['subreddit'] == 'GreatApes':
#                 posted_times_s.append(new_line['created_utc'])
#     #             except:  # avoid users with empty submission jsons
#     #                 print('user ' + author + ' has no submissions')
#     #                 continue
        file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_submissions.json"
        df_s = pd.read_csv(file_name)
        posted_time_s = df_s['created_utc']

        file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_comments.json"
        df_c = pd.read_csv(file_name)
        posted_time_c = df_c['created_utc']
#         with open(file_name,'r') as json_file:
#             for line in json_file:
#     #             try:
#                 new_line = json.loads(line)
#     #             if 'subreddit' in new_line.keys():
#     #                 if new_line['subreddit'] == 'GreatApes':
#                 posted_times_c.append(new_line['created_utc'])
#     #             except:  # avoid users with empty comment jsons
#     #                 print('user ' + author + ' has no comments')
#     #                 continue

    #     determine first post (submission or comment) in GreatApes
        if posted_times_s != [] and posted_times_c != []:  # user posted both submission and comment
            df_top200 = df_top200.append({"Username":author,"Time (First Post)":min([min(posted_times_s),min(posted_times_c)])},ignore_index=True)
        elif posted_times_s == [] and posted_times_c != []:                   # user only posted comment
            df_top200 = df_top200.append({"Username":author,"Time (First Post)":min(posted_times_c)},ignore_index=True)
        elif posted_times_c == [] and posted_times_s != []:                   # user only posted submission
            df_top200 = df_top200.append({"Username":author,"Time (First Post)":min(posted_times_s)},ignore_index=True)
        else:
            print('user ' + author + ' had no data in author file')

        if counter % 1000 == 0 and counter != 0:
            df_top200.to_csv(r'D:/ISI REU Python/Analyzed_Data/control_firstpost.csv',mode='a',header=False)
            df_top200 = pd.DataFrame(columns=['Username','Time (First Post)'])
            print('1000 authors processed')

        print(author + ' done')
        counter += 1
        
    except:
        print(author + ' NOT DONE!!!')
#         authors_error.append(author)
        error_again.append(author)

user mr_bacciagalupe had no data in author file
mr_bacciagalupe done
user oh_HELL_no had no data in author file
oh_HELL_no done
user jip- had no data in author file
jip- done
user married_fun had no data in author file
married_fun done
user shredster had no data in author file
shredster done
user imaliongrrr had no data in author file
imaliongrrr done
user Threwaway13570 had no data in author file
Threwaway13570 done
user abbcccddddd had no data in author file
abbcccddddd done
user AJGatherer had no data in author file
AJGatherer done
user pussysm0ke had no data in author file
pussysm0ke done
user Fenix7845 had no data in author file
Fenix7845 done
user doigotta11 had no data in author file
doigotta11 done
user kvnmahan had no data in author file
kvnmahan done
user eliavchik had no data in author file
eliavchik done
user swallowing_sadness had no data in author file
swallowing_sadness done
user MeanTweetCentral had no data in author file
MeanTweetCentral done
user The_Sky_Is_Orange had

In [23]:
df_top200

,Username,Time (First Post)


In [19]:
# df_top200.to_csv(r'D:/ISI REU Python/Analyzed Data/control_firstpost.csv',mode='w',header=True)

In [50]:
df_top200 = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/control_firstpost.csv')

In [51]:
datetimes = []

for index, row in df_top200.iterrows():
    datetimes.append(dt.datetime.fromtimestamp(row['Time (First Post)']))
    
df_top200['Datetime (First Post)'] = datetimes

In [58]:
df_top200['Date (First Post)'] = df_top200['Datetime (First Post)'].dt.date
df_top200

,Unnamed: 0,Username,Time (First Post),Datetime (First Post),Date (First Post)
0,0,aleos,1284619286,2010-09-16 01:41:26,2010-09-16
1,1,AmericanGal1996,1469300799,2016-07-23 14:06:39,2016-07-23
2,2,atomicocobutter,1497389160,2017-06-13 16:26:00,2017-06-13
3,3,Alienm00se,1343792685,2012-07-31 22:44:45,2012-07-31
4,4,AllThingsHockey,1436990218,2015-07-15 14:56:58,2015-07-15
...,...,...,...,...,...
479,479,biggiexo,1495679847,2017-05-24 21:37:27,2017-05-24
480,480,AntiAbleism,1482957456,2016-12-28 14:37:36,2016-12-28
481,481,a_horrible_person,1275399603,2010-06-01 08:40:03,2010-06-01
482,482,Barely_Intrepid,1425571339,2015-03-05 10:02:19,2015-03-05


In [59]:
df_top200.to_csv(r'D:/ISI REU Python/Analyzed Data/control_firstpost.csv',mode='w',header=True)

## determine time of GreatApes users' first post (in Reddit overall, not just in GreatApes)

In [25]:
gaauthor_list = []

In [27]:
with open(r'D:/ISI REU Python/PSAW Data/Full Data/GreatApes_authors.json','r') as json_file:  # open json of GreatApes users
    for line in json_file:
        gaauthor_list.append(line.replace('\n','').replace('"',''))
        
gaauthor_list = list(dict.fromkeys(gaauthor_list))
gaauthor_list.remove("[deleted]")

In [42]:
author_list = []

file_paths_s = []
for root, directories, files in os.walk(r'D:/ISI REU Python/Authors PSAW (Copy)/'):          # go through Authors PSAW folder to determine already-downloaded
    for filename in files:
        filepath = os.path.join(root, filename)
        if '_submissions.json' in filepath:
            new_filepath = filepath.replace(r'D:/ISI REU Python/Authors PSAW (Copy)/','').replace('_submissions.json','')
            file_paths_s.append(new_filepath)
file_paths_s = set(dict.fromkeys(file_paths_s))

# for author in gaauthor_list:
#     if author in file_paths:
#         author_list.append(author)

file_paths_c = []
for root, directories, files in os.walk(r'D:/ISI REU Python/Authors PSAW (Copy)/'):
    for filename in files:
        filepath = os.path.join(root, filename)
        if '_comments.json' in filepath:
            new_filepath = filepath.replace(r'D:/ISI REU Python/Authors PSAW (Copy)/','').replace('_comments.json','')
            file_paths_c.append(new_filepath)
file_paths_c = set(dict.fromkeys(file_paths_c))

author_list = list(set(gaauthor_list).intersection(file_paths_s.intersection(file_paths_c)))

In [28]:
# df_authors = pd.DataFrame(data={'Authors': gaauthor_list})

In [44]:
df_GreatApes = pd.DataFrame(columns = ['Username','Time (First Post)'])  # create dataframe

for author in author_list:
    
    posted_times_s = []
    posted_times_c = []

#     if the line in GreatApes, append to temporary times list
    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_submissions.json"
    with open(file_name,'r') as json_file:
        for line in json_file:
            try:
                new_line = json.loads(line)
#                 if 'subreddit' in new_line.keys():
#                     if new_line['subreddit'] == 'GreatApes':
                posted_times_s.append(new_line['created_utc'])
            except:  # avoid users with empty submission jsons
                print('user ' + author + ' has no submissions')
                continue

    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_comments.json"
    with open(file_name,'r') as json_file:
        for line in json_file:
            try:
                new_line = json.loads(line)
#                 if 'subreddit' in new_line.keys():
#                     if new_line['subreddit'] == 'GreatApes':
                posted_times_c.append(new_line['created_utc'])
            except:  # avoid users with empty comment jsons
                print('user ' + author + ' has no comments')
                continue

#     determine first post (submission or comment) in GreatApes
    if posted_times_s != [] and posted_times_c != []:  # user posted both submission and comment
        df_GreatApes = df_GreatApes.append({"Username":author,"Time (First Post)":min([min(posted_times_s),min(posted_times_c)])},ignore_index=True)
    elif posted_times_s == [] and posted_times_c != []:                   # user only posted comment
        df_GreatApes = df_GreatApes.append({"Username":author,"Time (First Post)":min(posted_times_c)},ignore_index=True)
    elif posted_times_c == [] and posted_times_s != []:                   # user only posted submission
        df_GreatApes = df_GreatApes.append({"Username":author,"Time (First Post)":min(posted_times_s)},ignore_index=True)
    else:
        print('user ' + author + ' had no data in author file')
    
    print(author + ' done')

carson378 done
disdatthrowaway2 done
user 1776isbest76 has no submissions
1776isbest76 done
user AssheadMiller has no submissions
AssheadMiller done
user SanDiegoHermit has no submissions
SanDiegoHermit done
saggytitties done
vi_warshawski done
TheCowsRComing done
user Idundidhatedemnegros has no submissions
Idundidhatedemnegros done
bleed_red_white_blue done
user dickjsteele has no submissions
dickjsteele done
user Hogarthy has no submissions
Hogarthy done
bmores8 done
OddsThatYouAreBlack done
rsashe1980 done
JuggerzTheCat done
thunder_c0ck done
Gail33 done
user CosbyIsRight has no submissions
CosbyIsRight done
agmaster done
Vertraggg done
aracistrabbit done
noumenon_ done
Wheelswhithinwheels done
TheOfficialPossum done
budguy68 done
ducknoises done
UnlimitedFlashbang done
Black-Muse done
Casey_04 done
Orange87 done
Iusedmyrealname done
Teller8 done
Dongacles done
lapollamasgrande done
user Kautkto has no comments
user Kautkto has no comments
user Kautkto has no comments
user Kautkto 

In [45]:
# df_GreatApes.to_csv(r'D:/ISI REU Python/Analyzed Data/treatment_firstpost.csv')

In [47]:
datetimes = []

for index, row in df_GreatApes.iterrows():
    datetimes.append(dt.datetime.fromtimestamp(row['Time (First Post)']))
    
df_GreatApes['Datetime (First Post)'] = datetimes

In [60]:
df_GreatApes['Date (First Post)'] = df_GreatApes['Datetime (First Post)'].dt.date
df_GreatApes

,Username,Time (First Post),Datetime (First Post),Date (First Post)
0,carson378,1328225565,2012-02-02 17:32:45,2012-02-02
1,disdatthrowaway2,1399400378,2014-05-06 13:19:38,2014-05-06
2,1776isbest76,1413245158,2014-10-13 19:05:58,2014-10-13
3,AssheadMiller,1548097027,2019-01-21 12:57:07,2019-01-21
4,SanDiegoHermit,1416541625,2014-11-20 21:47:05,2014-11-20
...,...,...,...,...
3524,POOP_FONDUE,1346063840,2012-08-27 05:37:20,2012-08-27
3525,u_fukn_wot_m8_,1406133395,2014-07-23 11:36:35,2014-07-23
3526,bangbangahah,1400795262,2014-05-22 16:47:42,2014-05-22
3527,ih8them,1407709818,2014-08-10 17:30:18,2014-08-10


In [61]:
df_GreatApes.to_csv(r'D:/ISI REU Python/Analyzed Data/treatment_firstpost.csv')

## match datetimes of first posts

In [62]:
df_top200

,Unnamed: 0,Username,Time (First Post),Datetime (First Post),Date (First Post)
0,0,aleos,1284619286,2010-09-16 01:41:26,2010-09-16
1,1,AmericanGal1996,1469300799,2016-07-23 14:06:39,2016-07-23
2,2,atomicocobutter,1497389160,2017-06-13 16:26:00,2017-06-13
3,3,Alienm00se,1343792685,2012-07-31 22:44:45,2012-07-31
4,4,AllThingsHockey,1436990218,2015-07-15 14:56:58,2015-07-15
...,...,...,...,...,...
479,479,biggiexo,1495679847,2017-05-24 21:37:27,2017-05-24
480,480,AntiAbleism,1482957456,2016-12-28 14:37:36,2016-12-28
481,481,a_horrible_person,1275399603,2010-06-01 08:40:03,2010-06-01
482,482,Barely_Intrepid,1425571339,2015-03-05 10:02:19,2015-03-05


In [63]:
df_match = df_GreatApes.merge(df_top200, on='Date (First Post)')

In [71]:
for i in ['Time (First Post)_x','Datetime (First Post)_x','Time (First Post)_y','Datetime (First Post)_y']:
    df_match = df_match.drop(i,axis=1)

In [73]:
df_match.drop(columns=df_match.columns[2], 
        axis=1, 
        inplace=True)

In [75]:
df_match.to_csv(r'D:/ISI REU Python/Analyzed Data/matched_firstpost.csv')

In [76]:
df_match

,Username_x,Date (First Post),Username_y
0,carson378,2012-02-02,AssuredlyAThrowAway
1,carson378,2012-02-02,Balthanos
2,disdatthrowaway2,2014-05-06,anubgek
3,thelongwaited,2014-05-06,anubgek
4,nupak,2014-05-06,anubgek
...,...,...,...
597,Mike12344321,2012-04-17,ArchangellePedophile
598,philosopherchaos,2013-12-05,bbk2800
599,supersouporsalad,2012-01-30,BetaThetaPirate
600,retrobuddha,2012-01-30,BetaThetaPirate


## find user join / first post date (OLD - DO NOT USE! USE FINAL_CONTROLGROUP2 INSTEAD)

In [2]:
file_paths = []

for root, directories, files in os.walk(r'D:/ISI REU Python/Authors PSAW (Copy)/'):
    for filename in files:
        filepath = os.path.join(root, filename)
        if '_comments.json' in filepath:
            new_filepath = filepath.replace(r'D:/ISI REU Python/Authors PSAW (Copy)/','').replace('_comments.json','')
            file_paths.append(new_filepath)

file_paths = list(dict.fromkeys(file_paths))

In [3]:
ga_authors = []

with open(r'D:/ISI REU Python/PSAW Data/Full Data/GreatApes_authors.json','r') as json_file:
    for line in json_file:
        ga_authors.append(line.replace('"','').replace('\n',''))
        
ga_authors = list(dict.fromkeys(ga_authors))
ga_authors.remove("[deleted]")

In [4]:
other_authors = []

for author in file_paths:
    if author not in ga_authors:
        other_authors.append(author)

In [5]:
len(other_authors)

3704

In [123]:
# df_other = pd.DataFrame(columns = ['Username','Time (First Post)'])  # create dataframe

def first_post(author):
    
    posted_times_s = []
    posted_times_c = []

#     if the line in GreatApes, append to temporary times list
#     try:
    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_submissions.json"
    with open(file_name,'r') as json_file:
        for line in json_file:
            try:
                new_line = json.loads(line)
                if 'subreddit' in new_line.keys():
#                     if new_line['subreddit'] != subreddit:
                    posted_times_s.append(new_line['created_utc'])
            except:  # avoid users with empty submission jsons
                print('user ' + author + ' has no submissions')
                continue
#     except:
#         print('u/' + author + ' has no submission file')

    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_comments.json"
    with open(file_name,'r') as json_file:
        for line in json_file:
            try:
                new_line = json.loads(line)
                if 'subreddit' in new_line.keys():
#                     if new_line['subreddit'] != subreddit:
                    posted_times_c.append(new_line['created_utc'])
            except:  # avoid users with empty comment jsons
                print('user ' + author + ' has no comments')
                continue

#     determine first post (submission or comment) in GreatApes
    if posted_times_s != [] and posted_times_c != []:  # user posted both submission and comment
        author_created.append(min([min(posted_times_s),min(posted_times_c)]))
#         df_other = df_other.append({"Username":author,"Time (First Post)":min([min(posted_times_s),min(posted_times_c)])},ignore_index=True)
    elif posted_times_s == [] and posted_times_c != []:                   # user only posted comment
        author_created.append(min(posted_times_c))
#         df_other = df_other.append({"Username":author,"Time (First Post)":min(posted_times_c)},ignore_index=True)
    elif posted_times_c == [] and posted_times_s != []:                   # user only posted submission
        author_created.append(min(posted_times_s))
#         df_other = df_other.append({"Username":author,"Time (First Post)":min(posted_times_s)},ignore_index=True)
    else:
        author_created.append(None)
        print('user ' + author + ' had no data in author file')
    
    print(author + ' done')

In [124]:
author_created = []

for author in ga_authors:
    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_comments.json"
    try:
        with open(file_name,'r') as json_file:
            for line in json_file:
    #                 try:
                new_line = json.loads(line)
                if 'author_created_utc' in new_line.keys():
                    if new_line['author_created_utc'] != None:
                        author_created.append(new_line['author_created_utc'])
                    else:
                        first_post(author)
                else:
                    first_post(author)
                break
#                 except:
#                     print('new_line error for u/' + author)
#                     continue
    except:
        author_created.append(None)
        print('error with u/' + author)

WoolenWolf done
zundish done
cabmerlot done
purple_monkey58 done
GodKingThoth done
MajSARS done
user scubaloon has no submissions
scubaloon done
sink_tinkler done
Parentoforphan done
Soviet_Russia321 done
Onyx_Blackmen done
nolo_me done
DeathRoux done
AMasonJar done
ioncehadsexinapool done
AnarchyFish done
DontGayMeBro done
CertifiedRabbi done
GipTukeMan done
blokesmunts done
WharGoul666 done
Icanus done
cainbackisdry done
JayStarr1082 done
canisliz done
Galton1911 done
I_am_a_mormon done
Aardopossadillo done
residude done
ScoochMagooch done
jayjayace done
Ironfist done
autoposting_system done
Randomonius done
myvirginityisstrong done
Quantumsheep0 done
secondshoah done
Opticalgangbang done
changemylife99 done
ROCKLOBSTER154 done
user Paganfun has no submissions
Paganfun done
arrestedfunk done
JakeDDrake done
zionvinman done
sambravers done
VY_Cannabis_Majoris done
martinskrtel done
conroy1 done
Saleroth done
imbeciI done
yolo_swag_tyme done
imjgaltstill done
user Free_Tokes has no sub

In [32]:
df_GreatApes.to_csv(r'D:/ISI REU Python/Analyzed Data/treatment_firstpost.csv')

In [128]:
print(len(ga_authors))
print(len(author_created))

3534
3534


In [129]:
# df_authorCreated = pd.DataFrame({"Username":other_authors,"Joined Reddit":author_created})
df_GreatApes_join = pd.DataFrame({"Username":ga_authors,"Joined Reddit":author_created})
df_GreatApes_join

,Username,Joined Reddit
0,Baryshnikov_Rifle,1.394732e+09
1,WoolenWolf,1.332825e+09
2,GoybeSmaap,1.437083e+09
3,zundish,1.396370e+09
4,pppk3125,1.425352e+09
...,...,...
3529,its-the-new-style,1.329483e+09
3530,gonjinetik,1.320295e+09
3531,XXXdrunkendonutsXXX,1.352270e+09
3532,rack49,1.275507e+09


In [125]:
df_authorCreated

,Username (Control),Date (First Post)
0,---CitationNeeded---,2015-02-06
1,--world,2017-08-01
2,--_--__--___--,2016-04-29
3,-917-,2016-07-15
4,-aytch-,2017-11-09
...,...,...
3699,_thattguy_,2017-11-04
3700,_y2b_,2011-07-28
3701,__Eudaimonia__,2016-05-20
3702,_____IsAMess,2016-07-04


In [126]:
# df_authorCreated.to_csv(r'D:/ISI REU Python/Analyzed Data/control_users.csv')

In [130]:
# create datetime column

datetimes = []

for index, row in df_GreatApes_join.iterrows():
    try:
        datetimes.append(datetime.datetime.fromtimestamp(row['Joined Reddit']))
    except:
        datetimes.append(None)
    
df_GreatApes_join['Datetime (First Post)'] = datetimes
df_GreatApes_join

,Username,Joined Reddit,Datetime (First Post)
0,Baryshnikov_Rifle,1.394732e+09,2014-03-13 10:38:52
1,WoolenWolf,1.332825e+09,2012-03-26 22:01:51
2,GoybeSmaap,1.437083e+09,2015-07-16 14:35:23
3,zundish,1.396370e+09,2014-04-01 09:39:57
4,pppk3125,1.425352e+09,2015-03-02 19:08:58
...,...,...,...
3529,its-the-new-style,1.329483e+09,2012-02-17 04:44:13
3530,gonjinetik,1.320295e+09,2011-11-02 21:37:58
3531,XXXdrunkendonutsXXX,1.352270e+09,2012-11-06 22:32:32
3532,rack49,1.275507e+09,2010-06-02 12:24:09


In [22]:
df_authorCreated.columns = ["Username (Control)","Datetime (First Post)"]

In [131]:
df_GreatApes_join['Date (First Post)'] = df_GreatApes_join['Datetime (First Post)'].dt.normalize()

In [132]:
df_GreatApes_join = df_GreatApes_join.drop(['Datetime (First Post)','Joined Reddit'],axis=1)

In [133]:
df_GreatApes_join.to_csv(r'D:/ISI REU Python/Analyzed Data/treatment_users_ga.csv')

In [134]:
df_GreatApes = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_firstpost.csv')

In [135]:
df_GreatApes['Datetime (First Post)'] = pd.to_datetime(df_GreatApes['Datetime (First Post)'])

In [136]:
df_GreatApes['Date (First Post)'] = df_GreatApes['Datetime (First Post)'].dt.normalize()

In [137]:
df_GreatApes = df_GreatApes.drop(['Datetime (First Post)','Time (First Post)'],axis=1)

In [138]:
df_lineup = pd.concat([df_GreatApes_join,df_authorCreated])
df_lineup

,Username,Date (First Post),Username (Control)
0,Baryshnikov_Rifle,2014-03-13,NaN
1,WoolenWolf,2012-03-26,NaN
2,GoybeSmaap,2015-07-16,NaN
3,zundish,2014-04-01,NaN
4,pppk3125,2015-03-02,NaN
...,...,...,...
3699,NaN,2017-11-04,_thattguy_
3700,NaN,2011-07-28,_y2b_
3701,NaN,2016-05-20,__Eudaimonia__
3702,NaN,2016-07-04,_____IsAMess


In [139]:
df_lineup = df_lineup.sort_values('Date (First Post)')

In [140]:
df_lineup.to_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_control-08-06-21.csv',index=False)

In [141]:
df_new = df_lineup.groupby(df_lineup['Date (First Post)']).aggregate({'Username':'first','Username (Control)':'first'})

In [142]:
df_new = df_new.dropna(axis=0,subset=['Username'])

In [143]:
df_new

,Username,Username (Control)
Date (First Post),,
2006-10-24,b34nz,None
2007-01-07,gordo65,None
2007-03-24,twistedcain,None
2007-06-11,terrortot,starfox863
2007-06-29,Tweakers,None
...,...,...
2016-02-07,4aredhead,aonf
2016-04-22,LindaRichmond,Broodfathers
2016-06-09,DisillusionedExLib,UltraTechLord


In [144]:
df_new.to_csv(r'D:/ISI REU Python/Analyzed Data/GreatApes_controlONLY-08-06-21.csv')

In [146]:
control_new = df_new['Username (Control)'].tolist()
control_new = list(dict.fromkeys(control_new))
control_new.remove(None)

In [149]:
print(control_new)
print(control)

['starfox863', 'nice_halibut', 'asmosdeus', '47toolate', 'kabukistar', 'glen_s', 'barrysandwich', 'sexyloser1128', 'raananh', 'sproket888', 'Arcon1337', 'Backupusername', 'cricket_guy', 'marvin_nash', 'BigBuz', 'amanitus', 'peetss', 'MuscleT', 'Ignix', 'Ambulant', 'MaunaLoona', 'BatmanV2', 'Brinner', 'accioreddit', 'thugl1fe', 'App1e5auce', 'biga29', 'ftw7969', 'BankerWhoLeavesAt420', 'bewareofmittens', 'Ham_on_Rye', 'BigMike__', 'andrewsj1', 'BIGlikeaBOSS', 'Alpha-as-fuck', 'thinkt4nk', 'redditvm', 'bluemax', 'alm_has', 'JustBuzzin', 'armedNINJA', 'AttractiveMeat', 'beer_madness', 'abigfatphoney', 'skepgineer', 'ArcadesRed', 'Ahnenglanz', 'Atmaweapon74', '09112001', 'antoine_chekov', 'AlanRosenthal', 'WiseCynic', 'bretw', 'davidverner', 'kono_hito_wa', 'pbtpu40', '15rthughes', 'WazzuMadBro', 'Akhaian', '_bani_', 'BaKdGoOdZ0203', 'Brassrain', 'AyleiDaedra', 'PonyIrl', 'Elmattador', 'PhoneySoprano', 'drunkai', '1337and0', 'Anal_Justice_League', 'Avery_Richman', 'weiss27md', 'bawlzsauce'

In [151]:
control_remaining = []

for i in control_new:
    if control_new not in control:
        control_remaining.append(i)
len(control_remaining)

645

In [153]:
hate_words = ['nigger','chimpout','dey','negroes','whitey','sheboon','nuffin','coon','groid','gibs','nig','tnb','faggot','ghetto','dindu','kike']

In [154]:
def word_counter(author, post):
    nword_counter = 0
    chimpout_counter = 0
    dey_counter = 0
    negroes_counter = 0
    whitey_counter = 0
    sheboon_counter = 0
    nuffin_counter = 0
    coon_counter = 0
    groid_counter = 0
    gibs_counter = 0
    nig_counter = 0
    tnb_counter = 0
    faggot_counter = 0
    ghetto_counter = 0
    dindu_counter = 0
    kike_counter = 0
#     hateword_counter = 0
    counter = 0
    global df_in
    global df_out
#     df_in = pd.DataFrame(columns=["Username","Time",'chimpout', 'dey', 'negroes', 'whitey', 'sheboon', 'nuffin', 'coon', 'groid', 'gibs', 'nig', 'tnb', 'faggot', 'ghetto', 'dindu', 'kike'])
#     df_out = pd.DataFrame(columns=["Username","Time",'chimpout', 'dey', 'negroes', 'whitey', 'sheboon', 'nuffin', 'coon', 'groid', 'gibs', 'nig', 'tnb', 'faggot', 'ghetto', 'dindu', 'kike'])
    
    if post == 'submission':
        if new_line['title'] != None:
            nword_counter += new_line['title'].count('nigger')
            chimpout_counter += new_line['title'].count('chimpout')
            dey_counter += new_line['title'].count('dey')
            negroes_counter += new_line['title'].count('negroes')
            whitey_counter += new_line['title'].count('whitey')
            sheboon_counter += new_line['title'].count('sheboon')
            nuffin_counter += new_line['title'].count('nuffin')
            coon_counter += new_line['title'].count('coon')
            groid_counter += new_line['title'].count('groid')
            gibs_counter += new_line['title'].count('gibs')
            nig_counter += new_line['title'].count('nig')
            tnb_counter += new_line['title'].count('tnb')
            faggot_counter += new_line['title'].count('faggot')
            ghetto_counter += new_line['title'].count('ghetto')
            dindu_counter += new_line['title'].count('dindu')
            kike_counter += new_line['title'].count('kike')
            counter += len(new_line['title'].split())
        if new_line['selftext'] != None:
            nword_counter += new_line['selftext'].count('nigger')
            chimpout_counter += new_line['selftext'].count('chimpout')
            dey_counter += new_line['selftext'].count('dey')
            negroes_counter += new_line['selftext'].count('negroes')
            whitey_counter += new_line['selftext'].count('whitey')
            sheboon_counter += new_line['selftext'].count('sheboon')
            nuffin_counter += new_line['selftext'].count('nuffin')
            coon_counter += new_line['selftext'].count('coon')
            groid_counter += new_line['selftext'].count('groid')
            gibs_counter += new_line['selftext'].count('gibs')
            nig_counter += new_line['selftext'].count('nig')
            tnb_counter += new_line['selftext'].count('tnb')
            faggot_counter += new_line['selftext'].count('faggot')
            ghetto_counter += new_line['selftext'].count('ghetto')
            dindu_counter += new_line['selftext'].count('dindu')
            kike_counter += new_line['selftext'].count('kike')
            counter += len(new_line['selftext'].split())

#         if 'created_utc' in new_line.keys():
#             df_out = df_out.append({"Username":author,"Time":datetime.datetime.fromtimestamp(new_line['created_utc']),"N-Word":nword_counter,"Chimpout":chimpout_counter,"Dey":dey_counter,"Negroes":negroes_counter,"Whitey":whitey_counter,"Sheboon":sheboon_counter,"Nuffin":nuffin_counter,"Coon":coon_counter,"Groid":groid_counter,"Gibs":gibs_counter,"Nig":nig_counter,"TNB":tnb_counter,"Faggot":faggot_counter,"Ghetto":ghetto_counter,"Dindu":dindu_counter,"Kike":kike_counter,"Total Words":counter},ignore_index=True)
    if post == 'comment':
        if new_line['body'] != None:
            nword_counter += new_line['body'].count('nigger')
            chimpout_counter += new_line['body'].count('chimpout')
            dey_counter += new_line['body'].count('dey')
            negroes_counter += new_line['body'].count('negroes')
            whitey_counter += new_line['body'].count('whitey')
            sheboon_counter += new_line['body'].count('sheboon')
            nuffin_counter += new_line['body'].count('nuffin')
            coon_counter += new_line['body'].count('coon')
            groid_counter += new_line['body'].count('groid')
            gibs_counter += new_line['body'].count('gibs')
            nig_counter += new_line['body'].count('nig')
            tnb_counter += new_line['body'].count('tnb')
            faggot_counter += new_line['body'].count('faggot')
            ghetto_counter += new_line['body'].count('ghetto')
            dindu_counter += new_line['body'].count('dindu')
            kike_counter += new_line['body'].count('kike')
            counter += len(new_line['body'].split())
            
    if 'created_utc' in new_line.keys():
        df_out = df_out.append({"Username":author,"Time":datetime.datetime.fromtimestamp(new_line['created_utc']),"N-Word":nword_counter,"Chimpout":chimpout_counter,"Dey":dey_counter,"Negroes":negroes_counter,"Whitey":whitey_counter,"Sheboon":sheboon_counter,"Nuffin":nuffin_counter,"Coon":coon_counter,"Groid":groid_counter,"Gibs":gibs_counter,"Nig":nig_counter,"TNB":tnb_counter,"Faggot":faggot_counter,"Ghetto":ghetto_counter,"Dindu":dindu_counter,"Kike":kike_counter,"Total Words":counter},ignore_index=True)

In [155]:
df_out = pd.DataFrame(columns=["Username","Time",'N-Word','Chimpout', 'Dey', 'Negroes', 'Whitey', 'Sheboon', 'Nuffin', 'Coon', 'Groid', 'Gibs', 'Nig', 'TNB', 'Faggot', 'Ghetto', 'Dindu', 'Kike','Total Words'])

df_out.to_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control (1).csv',index=False)

In [175]:
for author in control_remaining:
    
    if author in ['starfox863', 'nice_halibut', 'asmosdeus', '47toolate', 'kabukistar']:
        continue

#     df_in = pd.DataFrame(columns=["Username","Time",'Chimpout', 'Dey', 'Negroes', 'Whitey', 'Sheboon', 'Nuffin', 'Coon', 'Groid', 'Gibs', 'Nig', 'TNB', 'Faggot', 'Ghetto', 'Dindu', 'Kike'])
    df_out = pd.DataFrame(columns=["Username","Time",'N-Word','Chimpout', 'Dey', 'Negroes', 'Whitey', 'Sheboon', 'Nuffin', 'Coon', 'Groid', 'Gibs', 'Nig', 'TNB', 'Faggot', 'Ghetto', 'Dindu', 'Kike','Total Words'])
    
    file_name = r'D:/ISI REU Python/Authors PSAW (Copy)/'+author+'_submissions.json'
    try:
        with open(file_name,'r') as json_file:
            print('submissions for '+author+' starting...')
            for line in json_file:
                try:
                    new_line = json.loads(line)
                    word_counter(author,'submission')
                except:
                    print('user ' + author + ' has no submissions')
                    continue
    except:
        print('u/' + author + ' has no submission file')
    
    file_name = r'D:/ISI REU Python/Authors PSAW (Copy)/'+author+'_comments.json'
    with open(file_name,'r') as json_file:
        print('comments for '+author+' starting...')
        for line in json_file:
            try:
                new_line = json.loads(line)
                word_counter(author,'comment')
            except:
                print('user ' + author + ' has no comments')
                continue

    df_out.to_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control (1).csv',index=False,mode='a',header=False)
    
    print(author + ' done')
#     break

u/glen_s has no submission file
comments for glen_s starting...
glen_s done
submissions for barrysandwich starting...
comments for barrysandwich starting...
barrysandwich done
u/sexyloser1128 has no submission file
comments for sexyloser1128 starting...
sexyloser1128 done
u/raananh has no submission file
comments for raananh starting...
raananh done
u/sproket888 has no submission file
comments for sproket888 starting...
sproket888 done
submissions for Arcon1337 starting...
comments for Arcon1337 starting...
Arcon1337 done
submissions for Backupusername starting...
comments for Backupusername starting...
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments
user Backupusername has no comments


KeyboardInterrupt: 

In [157]:
# df_out.columns = ["Username (Control)","Time",'N-Word','Chimpout', 'Dey', 'Negroes', 'Whitey', 'Sheboon', 'Nuffin', 'Coon', 'Groid', 'Gibs', 'Nig', 'TNB', 'Faggot', 'Ghetto', 'Dindu', 'Kike','Total Words']

In [199]:
df_out = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control (1).csv')

In [200]:
df_out.columns = ["Username (Control)","Time",'N-Word','Chimpout', 'Dey', 'Negroes', 'Whitey', 'Sheboon', 'Nuffin', 'Coon', 'Groid', 'Gibs', 'Nig', 'TNB', 'Faggot', 'Ghetto', 'Dindu', 'Kike','Total Words']

ValueError: Length mismatch: Expected axis has 22 elements, new values have 19 elements

In [158]:
df_out = df_out.merge(df_new, on='Username (Control)')
# df_out = df_out.merge(df_out, on='Time')
# df_out = df_out.drop('Username_y',axis=1)

In [159]:
df_matched = df_out.merge(df_GreatApes, on='Username')

In [180]:
df_out1 = df_matched.drop(['Username','Date (First Post)','Time Difference (Days)','Time Difference (Months)'],axis=1)

In [196]:
# df_matched
# df_out.to_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control (1).csv',index=False,mode='a',header=False)
# df_out1.to_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control (1).csv',index=False,mode='a',header=False)
result_out = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control (1).csv')

index = result_out[result_out['N-Word'] == '2013-07-18 19:15:05'].index.values
for i in index:
    index_int = int(i)
    break
out_1 = result_out.iloc[:index_int,:]
out_2 = result_out.iloc[index_int:,:]

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [198]:
out_1.to_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control (1).csv')

In [161]:
df_matched['Time Difference (Days)'] = (pd.to_datetime(df_matched['Time']) - pd.to_datetime(df_matched['Date (First Post)'])).dt.days

In [162]:
df_matched['Time Difference (Months)'] = (df_matched['Time Difference (Days)']/30.436875).round(decimals=0)

In [163]:
df_matched

,Username (Control),Time,N-Word,Chimpout,Dey,Negroes,Whitey,Sheboon,Nuffin,Coon,...,TNB,Faggot,Ghetto,Dindu,Kike,Total Words,Username,Date (First Post),Time Difference (Days),Time Difference (Months)
0,kabukistar,2021-07-31 01:13:57,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6,hobdodgeries,2014-06-08,2610,86.0
1,kabukistar,2021-07-30 19:08:44,0,0,0,0,0,0,0,0,...,0,0,0,0,0,15,hobdodgeries,2014-06-08,2609,86.0
2,kabukistar,2021-07-30 18:06:56,0,0,0,0,0,0,0,0,...,0,0,0,0,0,17,hobdodgeries,2014-06-08,2609,86.0
3,kabukistar,2021-07-30 17:00:34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,74,hobdodgeries,2014-06-08,2609,86.0
4,kabukistar,2021-07-30 16:59:24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,17,hobdodgeries,2014-06-08,2609,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71254,kabukistar,2011-10-09 10:47:12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,198,hobdodgeries,2014-06-08,-973,-32.0
71255,kabukistar,2011-10-09 10:04:23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,31,hobdodgeries,2014-06-08,-973,-32.0
71256,kabukistar,2011-10-09 10:02:46,0,0,0,0,0,0,0,0,...,0,0,0,0,0,13,hobdodgeries,2014-06-08,-973,-32.0
71257,kabukistar,2011-10-09 09:57:28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,12,hobdodgeries,2014-06-08,-973,-32.0


In [164]:
df_matched.to_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control_matched (1).csv')

In [165]:
df_1 = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control_matched.csv')
df_2 = pd.read_csv(r'D:/ISI REU Python/Analyzed Data/hatewords_control_matched (1).csv')

In [169]:
df_matched = pd.concat([df_1, df_2], axis=0)
df_matched

,Unnamed: 0,Username (Control),Time,N-Word,Chimpout,Dey,Negroes,Whitey,Sheboon,Nuffin,...,TNB,Faggot,Ghetto,Dindu,Kike,Total Words,Username,Date (First Post),Time Difference (Days),Time Difference (Months)
0,0,BevoGenocide,2013-07-18 19:15:05,0,0,0,0,0,0,0,...,0,0,0,0,0,155,Invalid_Target,2013-07-17,1,0.0
1,1,BevoGenocide,2013-07-28 19:05:15,0,0,0,0,0,0,0,...,0,0,0,0,0,16,Invalid_Target,2013-07-17,11,0.0
2,2,BevoGenocide,2013-07-29 15:23:08,0,0,0,0,0,0,0,...,0,0,0,0,0,3,Invalid_Target,2013-07-17,12,0.0
3,3,BevoGenocide,2013-08-02 11:04:44,0,0,0,0,0,0,0,...,0,0,0,0,0,6,Invalid_Target,2013-07-17,16,1.0
4,4,BevoGenocide,2013-08-05 12:04:39,0,0,0,0,0,0,0,...,0,0,0,0,0,10,Invalid_Target,2013-07-17,19,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71254,71254,kabukistar,2011-10-09 10:47:12,0,0,0,0,0,0,0,...,0,0,0,0,0,198,hobdodgeries,2014-06-08,-973,-32.0
71255,71255,kabukistar,2011-10-09 10:04:23,0,0,0,0,0,0,0,...,0,0,0,0,0,31,hobdodgeries,2014-06-08,-973,-32.0
71256,71256,kabukistar,2011-10-09 10:02:46,0,0,0,0,0,0,0,...,0,0,0,0,0,13,hobdodgeries,2014-06-08,-973,-32.0
71257,71257,kabukistar,2011-10-09 09:57:28,0,0,0,0,0,0,0,...,0,0,0,0,0,12,hobdodgeries,2014-06-08,-973,-32.0


In [170]:
result_out = df_matched.drop(['Username (Control)','Username','Time','Date (First Post)','Time Difference (Days)'],axis=1)
result_out = result_out.groupby('Time Difference (Months)').sum()
result_out = result_out.reset_index()
# result_out = result_out.sort_values('Time Difference (Months)')
result_out

,Time Difference (Months),Unnamed: 0,N-Word,Chimpout,Dey,Negroes,Whitey,Sheboon,Nuffin,Coon,Groid,Gibs,Nig,TNB,Faggot,Ghetto,Dindu,Kike,Total Words
0,-32.0,25588260,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,7485
1,-31.0,31450820,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6789
2,-30.0,34751970,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6320
3,-29.0,30128760,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5722
4,-28.0,23096880,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,91.0,416969809,0,0,0,0,1,0,0,0,0,0,14,0,0,1,0,0,49449
124,92.0,370557571,0,0,0,0,0,0,0,0,0,0,11,0,1,0,0,0,46138
125,93.0,166255947,0,0,0,0,0,0,0,2,0,0,5,0,0,0,0,0,37645
126,94.0,136753023,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,36176


In [172]:
result_out['Average Proportion of Hate Words'] = ((result_out['Chimpout']+result_out['Dey']+result_out['Negroes']+result_out['Whitey']+result_out['Sheboon']+result_out['Nuffin']+result_out['Coon']+result_out['Groid']+result_out['Gibs']+result_out['Nig']+result_out['TNB']+result_out['Faggot']+result_out['Ghetto']+result_out['Dindu']+result_out['Kike']+result_out['N-Word']) * 100)/result_out['Total Words']

In [173]:
result_out

,Time Difference (Months),Unnamed: 0,N-Word,Chimpout,Dey,Negroes,Whitey,Sheboon,Nuffin,Coon,Groid,Gibs,Nig,TNB,Faggot,Ghetto,Dindu,Kike,Total Words,Average Proportion of Hate Words
0,-32.0,25588260,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,7485,0.026720
1,-31.0,31450820,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6789,0.014730
2,-30.0,34751970,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6320,0.015823
3,-29.0,30128760,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5722,0.017476
4,-28.0,23096880,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4819,0.020751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,91.0,416969809,0,0,0,0,1,0,0,0,0,0,14,0,0,1,0,0,49449,0.032357
124,92.0,370557571,0,0,0,0,0,0,0,0,0,0,11,0,1,0,0,0,46138,0.026009
125,93.0,166255947,0,0,0,0,0,0,0,2,0,0,5,0,0,0,0,0,37645,0.018595
126,94.0,136753023,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,36176,0.019350


In [174]:
result_out.to_csv(r'D:/ISI REU Python/Analyzed Data/hateword_control_combined.csv')

In [ ]:
# for utc in author_created:
#     gen_author_com = api.search_comments(author_created_utc=utc,
#                                         limit=1)
#     results_author_com = list(gen_author_com)

#     df = pd.DataFrame([obj.d_ for obj in results_author_com])
#     out = df.to_json(orient='records',lines=True)
#     exec("with open(r'D:/ISI REU Python/Authors PSAW (Copy)/"+a+"_comments.json','w+') as json_file: \n \t json_file.write(out)")

#     print(a + ' done')

In [3]:
# df_other = pd.DataFrame(columns = ['Username','Time (First Post)'])  # create dataframe

def first_post(author):
    
    posted_times_s = []
    posted_times_c = []

#     if the line in GreatApes, append to temporary times list
    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_submissions.json"
    with open(file_name,'r') as json_file:
        for line in json_file:
            try:
                new_line = json.loads(line)
                if 'subreddit' in new_line.keys():
                    if new_line['subreddit'] != 'GreatApes':
                        posted_times_s.append(new_line['created_utc'])
            except:  # avoid users with empty submission jsons
                print('user ' + author + ' has no submissions')
                continue

    file_name = "D:/ISI REU Python/Authors PSAW (Copy)/"+author+"_comments.json"
    with open(file_name,'r') as json_file:
        for line in json_file:
            try:
                new_line = json.loads(line)
                if 'subreddit' in new_line.keys():
                    if new_line['subreddit'] != 'GreatApes':
                        posted_times_c.append(new_line['created_utc'])
            except:  # avoid users with empty comment jsons
                print('user ' + author + ' has no comments')
                continue

#     determine first post (submission or comment) in GreatApes
    if posted_times_s != [] and posted_times_c != []:  # user posted both submission and comment
        df_other = df_other.append({"Username":author,"Time (First Post)":min([min(posted_times_s),min(posted_times_c)])},ignore_index=True)
    elif posted_times_s == [] and posted_times_c != []:                   # user only posted comment
        df_other = df_other.append({"Username":author,"Time (First Post)":min(posted_times_c)},ignore_index=True)
    elif posted_times_c == [] and posted_times_s != []:                   # user only posted submission
        df_other = df_other.append({"Username":author,"Time (First Post)":min(posted_times_s)},ignore_index=True)
    else:
        print('user ' + author + ' had no data in author file')
    
    print(author + ' done')

In [4]:
df_other['Datetime (First Post)'] = datetime.datetime.fromtimestamp(df_other['Time (First Post)'])

TypeError: cannot convert the series to <class 'int'>

In [ ]:
df_other = df_other.merge(df_GreatApes, on='Time (First Post)')

In [ ]:
df_other.sort_values('Username')

In [ ]:
# df_other.to_csv(r'D:/ISI REU Python/Analyzed Data/result_out_all.csv',index=False)